In [1]:
import os
import pandas as pd 
from ReviewSentence import ReviewSentence

In [2]:
dfs = []
versions = ['five', 'three']
for version in versions:
    source_dir = os.path.join('raw', version)
    for filename in os.listdir(source_dir):
        if filename == "Readme.txt":
            continue

        rows = []
        filepath = os.path.join(source_dir, filename)
        print("Processing %s.." % filepath)

        with open(filepath, encoding="utf-8") as lines:
            for ln in lines:
                line = ln.strip().replace("\t", " ")
                r = ReviewSentence.parse(line)
                if r is not None:
                    if r.sentence_type == 'review':
                        rows.append(r.to_row())
        df = pd.DataFrame.from_records(rows, columns=("content", "target"))
        df['filename'] = '-'.join([version, filename.replace('.txt', '')])
        dfs.append(df)
concat_df = pd.concat(dfs, ignore_index=True)

before_ = len(concat_df)
concat_df = concat_df.drop_duplicates(subset=['content'])
print('# of duplicates:', before_ - len(concat_df))

Processing raw/five/Creative Labs Nomad Jukebox Zen Xtra 40GB.txt..
Processing raw/five/Apex AD2600 Progressive-scan DVD player.txt..
Processing raw/five/Nikon coolpix 4300.txt..
Processing raw/five/Nokia 6610.txt..
Processing raw/five/Canon G3.txt..
Processing raw/three/Speaker.xml..
Processing raw/three/Router.xml..
Processing raw/three/Router.txt..
Processing raw/three/Speaker.txt..
Processing raw/three/Computer.xml..
Processing raw/three/Computer.txt..
# of duplicates: 49


In [1]:
concat_df.head()

NameError: name 'concat_df' is not defined

In [7]:
filepath = os.path.join('parsed', '%s_%d.json' % ('-'.join(versions), len(concat_df)))
concat_df.to_json(filepath)
print('Created: %s' % filepath)

Created: parsed/five-three_5995.json


In [8]:
filepath = os.path.join('parsed', '%s_%d.csv' % ('-'.join(versions), len(concat_df)))
concat_df.to_csv(filepath)
print('Created: %s' % filepath)

Created: parsed/five-three_5995.csv
